In [ ]:
#clear history
%reset -f

import numpy as np
np.random.seed(10)
import math

from enum import Enum
import networkx as nx
import matplotlib.pyplot as plt
from itertools import product
import random
import pandas as pd
#necessary for removing dublicates from a list of lists
import itertools


from mesa import Agent, Model
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid
from mesa.batchrunner import BatchRunner
import ast
import six


time_step_to_year = 52 
#import local ipnyb files
%run ./model.ipynb
%run ./agent.ipynb
%run ./model_output_functions.ipynb
%run ./batch_run_functions.ipynb

In [ ]:
from itertools import repeat
#https://stackoverflow.com/questions/6974695/python-process-pool-non-daemonic
import multiprocessing
# We must import this explicitly, it is not imported by the top-level
# multiprocessing module.
import multiprocessing.pool

class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [ ]:
def go_single(nexti, model_no,immunity_duration,migration_rate,\
              avg_contacted_agents,intervention_time,seed,date,birthdeath):
    "Runs a single model."
    if model_no == 1: #no competition (no MT)
        strain_list= []
        loci1=[0,1,2,3,4,5,6,7,8]
        for i in loci1:
            strain_list.append([i])
        no_loci_AT=1
        no_loci_MT=0
        n_infected_list=[15]*(len(strain_list))
    elif model_no == 2: #low 
        strain_list= []
        loci1=[0,1,2,3,4,5,6,7,8]
        loci3=[0,1,2]
        for i in loci1:
            for j in loci3:
                strain_list.append([i,j])
        no_loci_AT=1
        no_loci_MT=1
        n_infected_list=[5]*(len(strain_list))
    elif model_no == 3: #medium
        strain_list= []
        loci1=[0,1,2,3,4,5,6,7,8]
        loci3=[0,1]
        for i in loci1:
            for j in loci3:
                strain_list.append([i,j])
        no_loci_AT=1
        no_loci_MT=1
        n_infected_list=[7,8,8,7]*4+[7,8]
    elif model_no == 4: #high
        strain_list= []
        loci1=[0,1,2,3,4,5,6,7,8]
        for i in loci1:
            strain_list.append([i,0])
        no_loci_AT=1
        no_loci_MT=1
        n_infected_list=[15]*(len(strain_list))
    
    n_pop=2500
    
    prob_of_transmission_list=[0.05]*(len(strain_list))
    avg_recovery_time=4
    n_iterations=250 * time_step_to_year
    #no limit for max no infections
    max_no_infections=len(strain_list)
    birthdeath=birthdeath
    timeinterval_diversity=520
    carriage_percentage=0.1
    
    #RUN
    random.seed(1)
    np.random.seed(1)

    mymodel = BaseModel(agent_type = HumanBase, n_pop=n_pop,n_infected_list=n_infected_list, strain_list=strain_list,
                   avg_contacted_agents=avg_contacted_agents, 
                   prob_of_transmission_list=prob_of_transmission_list,
                   avg_recovery_time=avg_recovery_time,
                   n_iterations=n_iterations,
                   agentDataTime=0,
                  no_loci_AT=no_loci_AT,no_loci_MT=no_loci_MT,
                   max_no_infections=max_no_infections,
                   immunity_duration=immunity_duration,timeinterval_diversity=timeinterval_diversity,
                  migration_rate=migration_rate ,carriage_percentage=carriage_percentage,
                  birthdeath=birthdeath,seed = seed,intervention_time = intervention_time,
                   #is_homogeneous=False
                  )
   
    v = mymodel.datacollector.get_model_vars_dataframe()
    
    v['TimeStep'] = range(n_iterations+1)
    v = create_mean_cols(v,model_no)
    summary=pd.concat([v])
    #display(summary.head())
    #date = "InterventionTime_%s"%intervention_time
    summary.to_csv('Outputs/%s_%s_Model_%s_Imm_%s_Contacts_%s_IntTime_%s_BD_%s.csv'%(date,model_no,immunity_duration,\
                                                              avg_contacted_agents,intervention_time,birthdeath,\
                                                              nexti), index=False)

In [ ]:
"""
Only interventions parameter sets have been provided here.
For the runs in the no_intervention scenarios please consider:
    n_iterations = 150 * time_step_to_year (make a change in go_single function)
    birthdeath = [True,False]
    migration_rate = [0,0.0001]
    avg_contacted_agents = [5.5, 6.5, 7.5, 10, 12.5, 15, 20]
    immunity_duration = [0,2, 4, 15, 35, 40, 45, 150 * time_step_to_year ]
    model_no_list = [1,2,3,4]
"""

date = 'Interventions'
nrepeat = 10
nexti_list = list(range(nrepeat)) * 2
seed_list = [random.randint(0,99999) for i in range(nrepeat)] * 2
model_no_list = [1,4] * nrepeat 
intervention_time_list = [150*52] * nrepeat * 2

immunity_duration = 40
migration_rate = 0.0001
avg_contacted_agents = 20
birthdeath = True 

my_df = pd.DataFrame({
    'nexti': nexti_list,
    'seed': seed_list,
    'model_no': model_no_list,
    'intervention_time':intervention_time_list,
    'immunity_duration': immunity_duration * len(nexti_list),
    'migration_rate': migration_rate * len(nexti_list),
    'avg_contacted_agents': avg_contacted_agents * len(nexti_list),
    'birthdeath': birthdeath * len(nexti_list)
})

table1=render_mpl_table(my_df, header_columns=0, col_width=2.0)
table1.savefig('Outputs/ModelParameters_%s.png'%(date), dpi = 300,
                 bbox_inches='tight')



In [ ]:
#batch run
mypool = MyPool()
results=mypool.starmap(go_single,\
                       zip(nexti_list,model_no_list,repeat(immunity_duration),\
                           repeat(migration_rate),repeat(avg_contacted_agents),\
                           intervention_time_list,seed_list, repeat(date),\
                           repeat(birthdeath)))
mypool.close()
mypool.join()